## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-24-05-44-50 +0000,nyt,Pennsylvania Nursing Home Explosion Kills 2,https://www.nytimes.com/2025/12/23/us/explosio...
1,2025-12-24-05-30-45 +0000,nypost,Anna Kepner’s family’s ‘damage control’ texts ...,https://nypost.com/2025/12/24/us-news/anna-kep...
2,2025-12-24-05-05-39 +0000,startribune,Takeaways: Julius Randle steps up in fourth qu...,https://www.startribune.com/takeaways-julius-r...
3,2025-12-24-05-04-17 +0000,nypost,3 inmates who busted out of Georgia jail alleg...,https://nypost.com/2025/12/24/us-news/3-inmate...
4,2025-12-24-05-03-52 +0000,nyt,"Thailand, Attacking Cambodia, Says Its Target ...",https://www.nytimes.com/2025/12/24/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2420/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
132,trump,29
267,epstein,24
241,new,21
268,files,14
114,christmas,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
49,2025-12-24-01-08-43 +0000,bbc,"Trump trips, a fake video and 10 possible co-c...",https://www.bbc.com/news/articles/c5yj4yvx80do...,111
214,2025-12-23-14-57-01 +0000,nypost,Trump flew on Epstein’s private jet ‘at least ...,https://nypost.com/2025/12/23/us-news/trump-fl...,102
225,2025-12-23-13-05-22 +0000,nypost,Andrew Mountbatten-Windsor asked Ghislaine Max...,https://nypost.com/2025/12/23/world-news/andre...,91
162,2025-12-23-18-22-52 +0000,nypost,DOJ preemptively defends Trump as latest batch...,https://nypost.com/2025/12/23/us-news/justice-...,87
129,2025-12-23-20-29-39 +0000,latimes,Claims about Trump in Epstein files are 'untru...,https://www.latimes.com/politics/story/2025-12...,78


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
49,111,2025-12-24-01-08-43 +0000,bbc,"Trump trips, a fake video and 10 possible co-c...",https://www.bbc.com/news/articles/c5yj4yvx80do...
124,47,2025-12-23-20-42-27 +0000,latimes,"Supreme Court rules against Trump, bars Nation...",https://www.latimes.com/politics/story/2025-12...
3,38,2025-12-24-05-04-17 +0000,nypost,3 inmates who busted out of Georgia jail alleg...,https://nypost.com/2025/12/24/us-news/3-inmate...
90,35,2025-12-23-22-25-31 +0000,nypost,Zelensky warns Putin may be plotting Christmas...,https://nypost.com/2025/12/23/world-news/zelen...
229,33,2025-12-23-12-28-30 +0000,nypost,"Hunter Biden says he’s up to $15M in debt, cla...",https://nypost.com/2025/12/23/us-news/hunter-b...
82,33,2025-12-23-22-53-57 +0000,nyt,Libyan Military’s Chief of Staff and 4 Others ...,https://www.nytimes.com/2025/12/23/world/middl...
225,32,2025-12-23-13-05-22 +0000,nypost,Andrew Mountbatten-Windsor asked Ghislaine Max...,https://nypost.com/2025/12/23/world-news/andre...
1,32,2025-12-24-05-30-45 +0000,nypost,Anna Kepner’s family’s ‘damage control’ texts ...,https://nypost.com/2025/12/24/us-news/anna-kep...
216,32,2025-12-23-14-50-56 +0000,nypost,"Russian drone, missile attack on Ukraine kills...",https://nypost.com/2025/12/23/world-news/russi...
209,32,2025-12-23-15-19-04 +0000,nypost,Trump credits tariffs with boosting economy as...,https://nypost.com/2025/12/23/us-news/trump-cr...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
